## Modelo 2 - Active learning

Aqui, será criado um modelo para etiquetar as demais amostras, que não tiveram label atribuída manualmente (arquivom ./datasets/raw_data_with_labels.xlsx)

In [2]:
#importando pacotes
import pandas as pd
import numpy as np
import re
import time

import bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 100)

#https://strftime.org
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
#lendo dataframes
df_limpo = pd.read_pickle('./datasets/df_limpo.pkl')

In [4]:
df_limpo.head()

,data,views,title
0,2019-07-30,635,Finanças é Coisa de Criança!
1,2019-12-03,37305,Câncer 2020 - Profissional e Finanças 1° semestre
2,2017-03-30,98,Seja Rica: Conquiste sua Independência Financeira
3,2019-03-19,41,Independência FINANCEIRA
5,2020-01-21,389,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...


In [5]:
features = pd.read_pickle('./datasets/features.pkl')

In [6]:
features.head()

,views,views_por_dia
0,635,2.243816
1,37305,237.611465
2,98,0.086344
3,41,0.098558
5,389,3.601852


In [7]:
#carregando conjunto completo de dados
df = pd.read_excel('./datasets/raw_data_with_labels.xlsx', index_col=0)

In [8]:
#criando novo dataframe somente com dados etiquetadas
df_labeled = df[df.y.notnull()].copy()
df_labeled.shape

(503, 14)

In [9]:
#criando série com as labels
y = df_labeled['y'].copy()

In [10]:
#importando alguns modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [11]:
#criando sets de treino e validação, separando os dados em 50% (quantil 0.5)
mask_train = df_limpo.data < df_limpo.data.quantile(0.5)
mask_val = df_limpo.data >= df_limpo.data.quantile(0.5)

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]

In [12]:
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((250, 2), (253, 2), (250,), (253,))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title'] #coletando títulos do set de treino
title_val = df_limpo[mask_val]['title'] #coletando títulos do set de validação

title_vec = TfidfVectorizer(min_df = 2) #vetorizador. lembrar que modelos não trabalham com strings
#ocorrência mínima do termo em 2 vídeos

title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) #usamos somente transform para transformar somente baseado nas palavras no dataset passado para ele.

In [14]:
title_bow_train.shape  

(250, 211)

Mais sobre Tf iDF [aqui](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).
O resultado acima indica que foram criadas 211 colunas no title_bow_train. Cada coluna é um vetor para cada palavra do título. O formato será uma matriz esparsa (só armazena valores maiores que zero), do scipy.

In [15]:
title_bow_train

<250x211 sparse matrix of type '<class 'numpy.float64'>'
	with 1411 stored elements in Compressed Sparse Row format>

A matriz criada para title_bow_train armazenou 1411 valores na mmemória. Se fosse armazenar todos os valores, teríamos:

In [16]:
250 * 211

52750

Para juntar as features de texto com as numéricas, deve-se usar a função hstack do scipy.sparse (hstack e vstack permitem lidar com matrizes esparsas com mais eficiência.
Vamos juntas as features para poder treinar o novo modelo. Mais sobre scipy sparse [aqui](https://docs.scipy.org/doc/scipy/reference/sparse.html).

In [17]:
from scipy.sparse import hstack, vstack

In [18]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train]) #aqui, juntamos as features de treino em Xtrain com as geradas para os títulos das features de treino
Xval_wtitle = hstack([Xval, title_bow_val]) #ídem para validação

In [19]:
# o total de colunas será a soma das colunas das features numéricas com as de texto (2 + 211 = 213)
Xtrain_wtitle.shape, Xval_wtitle.shape

((250, 213), (253, 213))

In [20]:
%%time
#treinando modelo
# 1000 árvores

mdl = RandomForestClassifier(n_estimators = 1000, random_state = 0, class_weight="balanced", n_jobs=8)
mdl.fit(Xtrain_wtitle, ytrain)

Wall time: 1.22 s


RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=8,
                       random_state=0)

In [21]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [22]:
from sklearn.metrics import roc_auc_score, average_precision_score

Relembrando valores com árvore de decisão:
- average_precision_score = 0.4665
- roc_auc_score = 0.8083

Abaixo, as mesmas métricas para ramdom forest:

In [23]:
average_precision_score(yval, p)

0.7411156726976864

In [24]:
roc_auc_score(yval, p)

0.9096695226438188

queremos um mindf que melhore as métricas. mindf = 1 provou uma perda maior em ap do que a melhora em auc. vamos manter mindf = 2.

In [ ]:
mindf = 1 -> ap = 0.6864279965296931, auc = 0.9100979192166464
mindf = 2 -> ap = 0.7411156726976864, auc = 0.9096695226438188

### Active Learning

Em projetos reais, pode ser caro etiquetar muitos dados para treinar o modelo principal. Nesse exemplo, seguindo a mesma suposição do curso, vamos assumir que só há orçamento para etiquetar mais 100 exemplos. Nesse caso, uma boa prática é selecionar:
* 70 exemplos que o modelo tenha dificuldade de prever a resposta (~50% o valor da previsão)
* 30 exemplos aleatoriametnte  

Vamos carregar o dataframe df_unlabeled, com os dados que não foram etiquetados (y = null).

In [44]:
df_unlabeled = df[df.y.isnull()].dropna(how='all').copy() #parâmetro how='all' retira linhas com todas as colunas em branco

In [45]:
df_unlabeled.shape

(1005, 14)

In [46]:
df_unlabeled.tail()

,watch-title,y,watch-view-count,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0
1513,5 DICAS PARA COMPRAR MENOS | FINANÇAS DE A a Z...,NaN,23.299 visualizações,Publicado em 31 de out. de 2019,#Compras #Dinheiro #Consumo\n\n\n\n 5 DICAS...,23.299 visualizações\n\n\n\n\n\n\n\n2.863\n\nG...,https://i.ytimg.com/vi/zj3KFFwi--Q/maxresdefau...,1280.0,Já parou para pensar como você consome suas co...,1280.0,720.0,finanças com a nath,Educação,/channel/UCmhzmQBdXP9TeRtpQ-iWSPg
1514,QUANTO RENDE 10 MIL NO TESOURO DIRETO? Investi...,NaN,461.755 visualizações,Publicado em 23 de nov. de 2018,#QUANTO #RENDE #10MIL\n\n\n\n QUANTO RENDE ...,461.755 visualizações\n\n\n\n\n\n\n\n22.487\n\...,https://i.ytimg.com/vi/zjdcXx-og2k/maxresdefau...,1280.0,"Novo Curso Bolso Sem Dívidas, com desconto de ...",1280.0,720.0,economia,Educação,/channel/UCkvpo9jvRG73j7iCBTNL-OA
1515,5 Dicas para investir em Fundos de Investimentos,NaN,147.621 visualizações,Publicado em 13 de set. de 2016,5 Dicas para investir em Fundos de Investimentos,147.621 visualizações\n\n\n\n\n\n\n\n5.232\n\n...,https://i.ytimg.com/vi/ztTKCYH046A/maxresdefau...,1280.0,Você quer começar a investir no maravilhoso mu...,1280.0,720.0,mercado de capitais,Educação,/channel/UCVxYmVbcXxf-0HTJSlXW7yw
1516,Planejamento e finanças-Joyce Meyer,NaN,191.611 visualizações,Publicado em 10 de set. de 2018,Planejamento e finanças-Joyce Meyer,191.611 visualizações\n\n\n\n\n\n\n\n8.553\n\n...,https://i.ytimg.com/vi/zwagh_n_kkk/maxresdefau...,1280.0,,1280.0,720.0,Apocalipse,Filmes e desenhos,/channel/UCZQs2_GENrfNLLWRZZPkTzQ
1517,NaN,NaN,NaN,NaN,NaN,NaN,https://i.ytimg.com/vi/zwtEmh0Sqo0/hqdefault.jpg,480.0,🛑Inscreva-se em nosso canal do Telegram: https...,1280.0,720.0,NaN,NaN,NaN


Criando features para data de publicação e quantidade de views

In [30]:
#formato da data de publicação no dataframe com labels:
df_unlabeled['watch-time-text'].tail()

1513    Publicado em 31 de out. de 2019
1514    Publicado em 23 de nov. de 2018
1515    Publicado em 13 de set. de 2016
1516    Publicado em 10 de set. de 2018
1517                                NaN
Name: watch-time-text, dtype: object

In [31]:
map_mes = {'jan.':'Jan',
             'fev.':'Feb',
              'mar.':'Mar',
              'abr.':'Apr',
              'mai.':'May',
              'jun.':'Jun',
              'jul.':'Jul',
              'ago.':'Aug',
              'set.':'Sep',
              'out.':'Oct',
              'nov.':'Nov',
              'dez.':'Dec'}

In [32]:
def limpa_data (row):
    
    data_limpa = re.search(r"(\d+) de ([a-z]+)\. de (\d+)", row.loc['watch-time-text']).group()
    data_limpa = data_limpa.split('de ')
    data_limpa = [x.strip() for x in data_limpa]
    data_limpa[0] = ['0' + str(data_limpa[0]) if len(data_limpa[0]) == 1 else data_limpa[0]][0]
    data_limpa[1] = [v for k, v in map_mes.items() if k == data_limpa[1]][0]
    data = '-'.join(data_limpa)

    return data

In [47]:
# total de registros com a data nula
df_unlabeled[df_unlabeled['watch-time-text'].isna()].shape

(89, 14)

In [48]:
# as 89 linhas em branco não irão permitir criar as fetaures e serão removidas do dataset
df_unlabeled = df_unlabeled[df_unlabeled['watch-time-text'].notnull()].copy() #parâmetro how='all' retira linhas com todas as colunas em branco

In [49]:
df_unlabeled.head()

,watch-title,y,watch-view-count,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0
538,"Renda Passiva, Independência Financeira e Apos...",NaN,176 visualizações,Publicado em 28 de jan. de 2020,#RendaPassiva #LiberdadeFinanceira #RendaVariá...,176 visualizações\n\n\n\n\n\n\n\n46\n\nGostou ...,https://i.ytimg.com/vi/L3qXzNaqPAw/maxresdefau...,1280.0,Click aqui e se inscreva no canal: http://bit....,1280.0,720.0,Ricardo Soares,Educação,/channel/UCJXYGz3WKIjdmFS8GioMyBA
539,Conheça 6 INVESTIMENTOS que você pode fazer co...,NaN,37.342 visualizações,Publicado em 13 de jul. de 2019,Conheça 6 INVESTIMENTOS que você pode fazer co...,37.342 visualizações\n\n\n\n\n\n\n\n4.897\n\nG...,https://i.ytimg.com/vi/L7dejT5VbTw/maxresdefau...,1280.0,Conheça 6 INVESTIMENTOS que você pode fazer co...,1280.0,720.0,investindo com 100 reais,Pessoas e blogs,/channel/UCwLxXLLWEIJFHEeTMlYqHTA
540,Como Criar as Paginas do Site [ Dropshipping ],NaN,6 visualizações,Estreou em 5 de mai. de 2020,Como Criar as Paginas do Site [ Dropshipping ],6 visualizações\n\n\n\n\n\n\n\n1\n\nGostou des...,https://i.ytimg.com/vi/L935SV8Zm7w/maxresdefau...,1280.0,Conheça as minha estrategia exclusiva no Meu T...,1280.0,720.0,dropshipping 2020,Pessoas e blogs,/channel/UC9cZsG-Xdi9IgjPFzrZ2npw
541,INDEPENDÊNCIA FINANCEIRA (ESTRATÉGIA INCRÍVEL ...,NaN,2.519 visualizações,Publicado em 16 de fev. de 2019,INDEPENDÊNCIA FINANCEIRA (ESTRATÉGIA INCRÍVEL ...,2.519 visualizações\n\n\n\n\n\n\n\n347\n\nGost...,https://i.ytimg.com/vi/LHBsaft6EPU/maxresdefau...,1280.0,Como conquistar a independência financeira? Ne...,1280.0,720.0,erick,Educação,/channel/UCyCBeQuIVG09fgcNIEsYJgA
542,"Cartas Ciganas - TABULEIRO DA VIDA: Amor, Trab...",NaN,139.940 visualizações,Publicado em 14 de dez. de 2017,"Cartas Ciganas - TABULEIRO DA VIDA: Amor, Trab...",139.940 visualizações\n\n\n\n\n\n\n\n13.804\n\...,https://i.ytimg.com/vi/LJ_w29UlxRE/maxresdefau...,1280.0,Zap! - (79) 99142-2243,1280.0,720.0,tabuleiro cigano,Viagens e eventos,/channel/UC7-Y-xROMKq-DX65Xv4oOmA


In [50]:
df_unlabeled['data'] = df_unlabeled.apply(limpa_data, axis=1)

In [51]:
df_unlabeled.data.head()

538    28-Jan-2020
539    13-Jul-2019
540    05-May-2020
541    16-Feb-2019
542    14-Dec-2017
Name: data, dtype: object

In [53]:
#convertendo para data
df_unlabeled['data'] = pd.to_datetime(df_unlabeled['data'])

In [54]:
#criando novo df para features somente com o título e a data de publicação
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['watch-title']

In [55]:
df_limpo_u['data'] = df_unlabeled['data']

In [57]:
df_limpo_u.head()

,title,data
538,"Renda Passiva, Independência Financeira e Apos...",2020-01-28
539,Conheça 6 INVESTIMENTOS que você pode fazer co...,2019-07-13
540,Como Criar as Paginas do Site [ Dropshipping ],2020-05-05
541,INDEPENDÊNCIA FINANCEIRA (ESTRATÉGIA INCRÍVEL ...,2019-02-16
542,"Cartas Ciganas - TABULEIRO DA VIDA: Amor, Trab...",2017-12-14


In [58]:
#adicionando coluna com view count formatado
df_limpo_u['views'] = df_unlabeled['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)

In [59]:
df_limpo_u.head()

,title,data,views
538,"Renda Passiva, Independência Financeira e Apos...",2020-01-28,176
539,Conheça 6 INVESTIMENTOS que você pode fazer co...,2019-07-13,37342
540,Como Criar as Paginas do Site [ Dropshipping ],2020-05-05,6
541,INDEPENDÊNCIA FINANCEIRA (ESTRATÉGIA INCRÍVEL ...,2019-02-16,2519
542,"Cartas Ciganas - TABULEIRO DA VIDA: Amor, Trab...",2017-12-14,139940


In [60]:
df_limpo_u.data.quantile(.5)

Timestamp('2019-07-24 00:00:00')

In [87]:
# criando dataframe para as features unlabeled:
features_u = pd.DataFrame(index=df_limpo_u.index)

In [88]:
#criando coluna com dias que se passaram desde a publicação até a data referência
features_u['dias_publicado'] = (pd.to_datetime("2020-05-08") - df_limpo_u['data']) / np.timedelta64(1, 'D') #denominador cria um objeto timedelta do numpy em diferença de 1 dia

#adicionando a coluna de views ao dataframe features
features_u['views'] = df_limpo_u['views']

#calculando views por dia
features_u['views_por_dia'] = features_u['views']/features_u['dias_publicado']

# retirando coluna de dias após publicação (para explicação, ver notebook para modelo1)
features_u = features_u.drop('dias_publicado', axis = 1)

In [90]:
#verificando se há dados np.inf
features_u[features_u['views_por_dia'].isin([np.nan, np.inf, -np.inf])]

,views,views_por_dia
1130,1,inf


In [93]:
#linha 1130 em df_limpo_u
df_limpo_u.loc[1130, :]

title    💥Designer de Sobrancelhas AULA Grátis (MARCAÇÃ...
data                                   2020-05-08 00:00:00
views                                                    1
Name: 1130, dtype: object

O problema ocorreu justamente por que o vídeo foi publicado à meia noite da data referência. Por isso o valor e dias_publicado  = 0 e views por dia foi para infinito. Será preciso deletar essa linha do df_limpo_u.

In [94]:
#resolvendo problema de valores np.inf (mais aqui https://numpy.org/devdocs/reference/constants.html#numpy.inf)
features_u = features_u[~features_u['views_por_dia'].isin([np.nan, np.inf, -np.inf])].copy()

In [96]:
df_limpo_u = df_limpo_u.drop(1130).copy()

In [105]:
df_unlabeled = df_unlabeled.drop(1130).copy()

In [107]:
features_u.shape, df_limpo_u.shape, df_unlabeled.shape

((915, 2), (915, 3), (915, 15))

In [81]:
features_u.head()

,views,views_por_dia
538,176,1.742574
539,37342,124.473333
540,6,2.000000
541,2519,5.635347
542,139940,159.748858


Aqui, iremos usar a random forest treinada anteriormente para prever quais dos exemplos em features_u que o modelo está com dificuldade.

In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_u = df_limpo_u['title'] 
title_bow_u = title_vec.transform(title_u)

In [99]:
title_bow_u

<915x211 sparse matrix of type '<class 'numpy.float64'>'
	with 4359 stored elements in Compressed Sparse Row format>

In [100]:
# função hstack para juntas as fetures
Xu_wtitle = hstack([features_u, title_bow_u])

In [101]:
Xu_wtitle

<915x213 sparse matrix of type '<class 'numpy.float64'>'
	with 6171 stored elements in COOrdinate format>

In [102]:
#criando vetor com pesos das previsões para o valor positivo
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [108]:
#adicionando a coluna com 'probabilidade' previsões ao df_unlabeled
# "probabilidade" pq na prática, não é a probabildiade matemática, já que o modelo está ponderando as previsões de forma distinta devido ao desbalanceio das 
#classes
df_unlabeled['p'] = pu

In [109]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0,data,p
538,"Renda Passiva, Independência Financeira e Apos...",NaN,176 visualizações,Publicado em 28 de jan. de 2020,#RendaPassiva #LiberdadeFinanceira #RendaVariá...,176 visualizações\n\n\n\n\n\n\n\n46\n\nGostou ...,https://i.ytimg.com/vi/L3qXzNaqPAw/maxresdefau...,1280.0,Click aqui e se inscreva no canal: http://bit....,1280.0,720.0,Ricardo Soares,Educação,/channel/UCJXYGz3WKIjdmFS8GioMyBA,2020-01-28,0.16


In [118]:
# Criando máscara para filtrar somente registros "difíceis" para o modelo
mask_u = (df_unlabeled['p'] >= 0.45) & (df_unlabeled['p'] <= 0.55)

In [119]:
# total de registros na faixa. A máscara é um vetor booleano com True para valores de p na faixa determinada.
mask_u.sum()

66

In [122]:
#como a faixa determinada já retorna uma amostra próxima de 70 exemplos, vamos mantê-la
df_unlabeled[mask_u].head()

,watch-title,y,watch-view-count,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0,data,p
566,TOP 10 AÇÕES PARA INVESTIR EM MAIO DE 2020 - ...,NaN,772 visualizações,Publicado em 2 de mai. de 2020,TOP 10 AÇÕES PARA INVESTIR EM MAIO DE 2020 - ...,772 visualizações\n\n\n\n\n\n\n\n27\n\nGostou ...,https://i.ytimg.com/vi/M8MSZipIWyg/maxresdefau...,1280.0,TOP 10 AÇÕES PARA INVESTIR EM MAIO DE 2020 - A...,1280.0,720.0,COMO INVESTIR,Pessoas e blogs,/channel/UCL6FUaPa_l8MAzrk1oqxCFw,2020-05-02,0.548
579,Qual é o melhor Investimento? Comprar imóveis ...,NaN,2.555 visualizações,Estreou em 5 de mai. de 2020,Qual é o melhor Investimento? Comprar imóveis ...,2.555 visualizações\n\n\n\n\n\n\n\n454\n\nGost...,https://i.ytimg.com/vi/MhUjrYvJXrw/maxresdefau...,1280.0,Separei outras Dicas dentro do E-book de FII's...,1280.0,720.0,passiva,Licença de atribuição Creative Commons (reutil...,/channel/UCSXdP8V4jRaw-8YMTpa6glw,2020-05-05,0.514
586,ESTABILIDADE x PROSPERIDADE 💲💲 - Pablo Marçal ...,NaN,64.519 visualizações,Publicado em 22 de ago. de 2019,ESTABILIDADE x PROSPERIDADE 💲💲 - Pablo Marçal ...,64.519 visualizações\n\n\n\n\n\n\n\n8.915\n\nG...,https://i.ytimg.com/vi/Mt7vb3D6-TM/maxresdefau...,1280.0,Até quando você vai ser escravo do dinheiro? C...,1280.0,720.0,dinheiro,Pessoas e blogs,/channel/UCbroBIg8zvIH8-F4631wJhA,2019-08-22,0.457
595,Direito Constitucional Finanças Públicas e Orç...,NaN,881 visualizações,Publicado em 28 de abr. de 2019,Direito Constitucional Finanças Públicas e Orç...,881 visualizações\n\n\n\n\n\n\n\n32\n\nGostou ...,https://i.ytimg.com/vi/NLXfZmDBFOQ/hqdefault.jpg,480.0,Direito Constitucional é o ramo do direito púb...,640.0,360.0,Aulas para concursos,Pessoas e blogs,/channel/UCgMMBBX9mef8QpHMubFDh3g,2019-04-28,0.450
621,Banco central derruba juros - Corte na Selic -...,NaN,390 visualizações,Publicado em 6 de mai. de 2020,Banco central derruba juros - Corte na Selic -...,390 visualizações\n\n\n\n\n\n\n\n88\n\nGostou ...,https://i.ytimg.com/vi/OWQwknATc7A/maxresdefau...,1280.0,ERRATA * NO VÍDEO FALO QUE O COPOM SE REÚNE DE...,1280.0,720.0,NaN,Pessoas e blogs,/channel/UCfPKiDwwCqdKHIv5WZxUwXA,2020-05-06,0.459


Agora, precisamos selecionar exemplos aleatórios entre aqueles que não estão na faixa de indecisão determinada.

In [123]:
#df com os exemplos difíceis para o modelo
dificeis = df_unlabeled[mask_u]

In [125]:
#df com dados aleatórios
aleatorios = df_unlabeled[~mask_u].sample(34, random_state=0) #100 - 66 = 34

In [126]:
#salvando essas 100 amostras em uma planilha para etiquetá-los e retorná-los para o modelo
pd.concat([dificeis, aleatorios]).to_excel("./datasets/active_label.xlsx", engine='xlsxwriter')